In [2]:
from PIL import Image
import os 
import tifffile as tf
import numpy as np
import pandas as pd
#读取tif文件

In [3]:
path = 'D:\Data\TRIMS_Aqua_TP/2008/2008\DAY'
path_gldd = r'D:/CSSP/SJY/sanjiangyuanmask.tif'
storagename = 'SJYsurface_TG1.csv'
mask_tf = tf.imread(path_gldd)
filtered_data = []
mask = mask_tf != 0  # 将掩码转换为布尔数组，非零值视为True
l1 = os.listdir(path)
l1.sort() 
l1 = l1[:366]
sum = 0
count = 0
ave = 0 
tg_daily = []
result_df = pd.DataFrame() #结果数组
for file_name in l1:
        file_path = os.path.join(path, file_name)
        # 直接读取文件，无需使用with语句
        data = tf.imread(file_path)
        gldd_lst = np.array(data[1145, 1661])
        print( gldd_lst)

        # 使用掩码过滤数据并计算总和及计数
        filtered_data.append(gldd_lst) 
        #file_sum = filtered_data.sum()
        #file_sum = np.array(file_sum)        

        # 累加到全局变量


filtered_data = np.array(filtered_data)
filtered_data.to_csv(storagename, index=False)
# 计算全局平均值，避免除以零
global_average = sum / count if count else np.nan
print("全局平均值:", global_average)            

25600
25600
25700
25600
25600
25600
25600
25600
25600
25600
25600
25600
25600
25600
25600
25600
25800
25700
25800
25600
25600
25600
25800
25800
25600
25800
25600
25600
25600
25700
25700
25600
25600
25600
25600
25600
25600
25600
25600
25600
25600
25600
25600
25600
25600
25700
25600
25700
25800
25600
25600
25600
25700
25700
25700
25600
25700
25600
25600
25800
25600
25900
25900
25800
25700
25500
25800
25800
25800
25900
25800
25900
26000
26100
25900
25800
25800
25900
25700
25800
25700
25800
26000
26000
26100
26000
26100
26200
26100
26000
25800
26000
25900
26000
26000
26400
26300
26300
26300
26300
26300
26000
26300
26100
26000
26400
26400
26400
26400
26400
26400
26200
26500
26700
26800
26700
26800
26600
26800
26700
26800
26800
26900
26800
26700
26900
26800
26800
26900
26900
27000
26800
26900
26900
26900
26900
27000
26600
26900
26300
26600
26900
26900
26900
26900
27000
27000
27000
26900
26900
26900
26800
26800
26900
26900
26900
26900
27000
27000
27200
27200
27100
26900
27200
27200
26800
2710

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [13]:
glacier_elevation = 'D:\download\dhdt/N33E091_2000-01-01_2020-01-01_dhdt.tif'
mask = 'D:\download\dhdt/N33E091_2000-01-01_2020-01-01_dhdt.tif'
glc_elv = tf.imread(glacier_elevation)
mask_glc = tf.imread(glacier_elevation)
mask = mask_glc != -9999.
GLDD_elv = glc_elv[mask]
GLDD_elv_mean = np.nanmean(GLDD_elv)


In [6]:
import os
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
import pyproj
import numpy as np


def process_tiff_files(year_range, base_path, min_lon, min_lat, max_lon, max_lat, storagename):
    result_df = pd.DataFrame()  # 结果数组
    
    for year in year_range:
        Var_daily = []  # 日平均降雨量储存数组
        path = f'{base_path}/{year}/{year}/NIGHT'
        try:
            file_list = os.listdir(path)
        except FileNotFoundError:
            print(f"Directory for year {year} not found, skipping.")
            continue

        file_list.sort()
        file_list = file_list[:366]
        
        # 读取并处理每个TIFF文件
        for file_name in file_list:
            file_path = os.path.join(path, file_name)
            tif_file = file_path

            with rasterio.open(tif_file) as dataset:
                # 获取数据数组
                data = dataset.read(1)  # 读取第一波段的数据

                # 获取地理信息
                crs = dataset.crs  # 投影信息
                transform = dataset.transform  # 仿射变换矩阵

                # 使用pyproj将经纬度范围转换为栅格坐标
                transformer = pyproj.Transformer.from_crs('EPSG:4326', crs, always_xy=True)
                min_x, min_y = transformer.transform(min_lon, min_lat)
                max_x, max_y = transformer.transform(max_lon, max_lat)

                # 创建多边形掩膜
                bbox = box(min_x, min_y, max_x, max_y)
                geo = gpd.GeoDataFrame({'geometry': [bbox]}, crs=crs)

                # 掩膜数据
                out_image, out_transform = mask(dataset, geo.geometry, crop=True)
                out_image = out_image[0]  # 由于只有一个波段，获取第一个波段的数据

                # 计算out_image的平均值
                valid_data = out_image[out_image != dataset.nodata]  # 排除无效数据
                mean_value = np.mean(valid_data) if valid_data.size > 0 else np.nan

                # 打印结果
                print(f"Processed {file_name} for year {year}")
                print(f"Mean value of the masked area: {mean_value}")
                Var_daily.append(mean_value)
        
        # 保存结果到文件
        df = pd.DataFrame(Var_daily, columns=[year])  # 数据转为DataFrame
        result_df = pd.concat([result_df, df], axis=1)  # 拼接
                
    result_df.columns = year_range
    result_df.to_csv(storagename, index=False)  # 保存结果到CSV文件            


# 设置参数
year_range = range(2008, 2009)
base_path = 'D:/Data/TRIMS_Aqua_TP'
min_lon, min_lat =88.665,30.635
max_lon, max_lat = 103.065,37.535
storagename = 'daily_TG_average_night.csv'



In [7]:
process_tiff_files(year_range, base_path, min_lon, min_lat, max_lon, max_lat, storagename)

Processed TRIMS_Aqua2008001D.tif for year 2008
Mean value of the masked area: 27868.40654171236
Processed TRIMS_Aqua2008002D.tif for year 2008
Mean value of the masked area: 27880.50385494958
Processed TRIMS_Aqua2008003D.tif for year 2008
Mean value of the masked area: 27811.486947334633
Processed TRIMS_Aqua2008004D.tif for year 2008
Mean value of the masked area: 27807.35535524826
Processed TRIMS_Aqua2008005D.tif for year 2008
Mean value of the masked area: 27703.27384295789
Processed TRIMS_Aqua2008006D.tif for year 2008
Mean value of the masked area: 27763.389794581006
Processed TRIMS_Aqua2008007D.tif for year 2008
Mean value of the masked area: 27793.488943349294
Processed TRIMS_Aqua2008008D.tif for year 2008
Mean value of the masked area: 27930.779073162685
Processed TRIMS_Aqua2008009D.tif for year 2008
Mean value of the masked area: 27947.328331594763
Processed TRIMS_Aqua2008010D.tif for year 2008
Mean value of the masked area: 27852.530810620447
Processed TRIMS_Aqua2008011D.tif f

In [10]:
import os
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.enums import Resampling
import pyproj
import geopandas as gpd
from shapely.geometry import box
import netCDF4 as nc
from tqdm import tqdm

def process_tiff_files2nc(year_range, base_path, mask_file, storagename, varname):
    with rasterio.open(mask_file) as dataset:
        # 获取掩膜的边界和数据
        mask = dataset.read(1)  # 读取第一波段的数据
        transform = dataset.transform  # 获取仿射变换矩阵
        bounds = dataset.bounds  # 获取边界
        crs = dataset.crs  # 获取投影信息

    # 将掩膜的地理边界转换为栅格坐标
    transformer = pyproj.Transformer.from_crs('EPSG:4326', crs, always_xy=True)
    min_lon, min_lat = bounds.left, bounds.bottom
    max_lon, max_lat = bounds.right, bounds.top
    min_x, min_y = transformer.transform(min_lon, min_lat)
    max_x, max_y = transformer.transform(max_lon, max_lat)

    for year in year_range:
        Var_daily = []  # 日平均降雨量储存数组
        path = f'{base_path}/{year}/{year}/DAY'
        try:
            file_list = os.listdir(path)
        except FileNotFoundError:
            print(f"Directory for year {year} not found, skipping.")
            continue

        file_list.sort()
        file_list = file_list[:366]
        
        # 读取并处理每个TIFF文件
        for file_name in file_list:
            file_path = os.path.join(path, file_name)
            print(f"Processing {file_path}")

            with rasterio.open(file_path) as dataset:
                # 获取数据数组
                data = dataset.read(1)  # 读取第一波段的数据
                print(data.shape)
                # 使用掩膜裁剪数据
                bbox = box(min_x, min_y, max_x, max_y)
                geo = gpd.GeoDataFrame({'geometry': [bbox]}, crs=crs)
                out_image, out_transform = mask(dataset, geo.geometry, crop=True)
                out_image = out_image[0]  # 由于只有一个波段，获取第一个波段的数据
                
                Var_daily.append(out_image)  # 将数据添加到列表中

    # 将所有 TIFF 数据堆叠并计算平均值
    Var_daily = np.array(Var_daily)
    Var_ave = np.nanmean(Var_daily, axis=0)  # 计算每个网格点的多年平均值
        
    # 生成 NetCDF 文件
    with nc.Dataset(storagename, 'w', format='NETCDF4') as dst:
        lat_dim = dst.createDimension('lat', Var_ave.shape[0])
        lon_dim = dst.createDimension('lon', Var_ave.shape[1])

        # 创建纬度和经度变量
        latitudes = np.linspace(min_lat, max_lat, Var_ave.shape[0])
        longitudes = np.linspace(min_lon, max_lon, Var_ave.shape[1])
        
        latitudes_var = dst.createVariable('latitude', np.float32, ('lat',))
        longitudes_var = dst.createVariable('longitude', np.float32, ('lon',))
        
        latitudes_var[:] = latitudes
        longitudes_var[:] = longitudes

        # 创建存储数据的变量
        mean_var = dst.createVariable(varname, np.float32, ('lat', 'lon',))
        mean_var[:, :] = Var_ave / 100  # 归一化处理，假设数据需要处理

        mean_var.long_name = f"Mean {varname} from {year_range[0]} to {year_range[-1]}"
    
    print(f"Data saved to {storagename}")

base_path = 'D:/Data/TRIMS_Aqua_TP'
mask_file = 'D:\CSSP\SJY\mask_TGSJY6.tif'
year_range = range(2001,2002)
varname = 'TG'
storagename = 'daily_TG_average_night.nc'
process_tiff_files2nc(year_range, base_path, mask_file, storagename, varname)






Processing D:/Data/TRIMS_Aqua_TP/2001/2001/DAY\TRIMS_Aqua2001001D.tif
(2772, 2971)


TypeError: 'numpy.ndarray' object is not callable

In [9]:
import os
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.features import shapes
import netCDF4 as nc
from tqdm import tqdm

def process_tiff_files2nc(year_range, base_path, mask_file, storagename, varname):
    with rasterio.open(mask_file) as dataset:
        # 获取掩膜文件的数据
        mask_data = dataset.read(1)  # 读取第一波段的掩膜
        transform = dataset.transform  # 获取仿射变换矩阵
        crs = dataset.crs  # 获取投影信息

        # 将掩膜的非零值转换为多边形
        mask_shapes = list(shapes(mask_data, transform=transform))
        print(mask_data.shape)
    for year in year_range:
        Var_daily = []  # 日平均降雨量储存数组
        path = f'{base_path}/{year}/{year}/DAY'
        try:
            file_list = os.listdir(path)
        except FileNotFoundError:
            print(f"Directory for year {year} not found, skipping.")
            continue

        file_list.sort()
        file_list = file_list[:366]
        
        # 读取并处理每个TIFF文件
        for file_name in tqdm(file_list, desc=f"Processing year {year}"):
            file_path = os.path.join(path, file_name)

            with rasterio.open(file_path) as dataset:
                # 获取数据数组
                data = dataset.read(1)  # 读取第一波段的数据
               
                # 使用掩膜的多边形裁剪数据
                #out_image, _ = mask(data , [shape[0] for shape in mask_shapes], crop=True)
                out_image = data[647:1544,1388:2770]
                Var_daily.append(out_image)  # 将裁剪后的数据添加到列表中

    # 将所有 TIFF 数据堆叠并计算平均值
    Var_daily = np.array(Var_daily)
    Var_ave = np.nanmean(Var_daily, axis=0)  # 计算每个网格点的多年平均值
        
    # 生成 NetCDF 文件
    with nc.Dataset(storagename, 'w', format='NETCDF4') as dst:
        lat_dim = dst.createDimension('lat', Var_ave.shape[0])
        lon_dim = dst.createDimension('lon', Var_ave.shape[1])

        # 创建纬度和经度变量
        latitudes = np.linspace(transform[5], transform[5] + transform[4] * Var_ave.shape[0], Var_ave.shape[0])
        longitudes = np.linspace(transform[2], transform[2] + transform[0] * Var_ave.shape[1], Var_ave.shape[1])
        
        latitudes_var = dst.createVariable('latitude', np.float32, ('lat',))
        longitudes_var = dst.createVariable('longitude', np.float32, ('lon',))
        
        latitudes_var[:] = latitudes
        longitudes_var[:] = longitudes

        # 创建存储数据的变量
        mean_var = dst.createVariable(varname, np.float32, ('lat', 'lon',))
        mean_var[:, :] = Var_ave / 100  # 归一化处理，假设数据需要处理

        mean_var.long_name = f"Mean {varname} from {year_range[0]} to {year_range[-1]}"
    
    print(f"Data saved to {storagename}")


# 使用示例
base_path = 'D:/Data/TRIMS_Aqua_TP'
mask_file = 'D:/CSSP/SJY/mask_TGSJY6.tif'
year_range = range(2001, 2002)
varname = 'TG'
storagename = 'daily_TG_average_night.nc'
process_tiff_files2nc(year_range, base_path, mask_file, storagename, varname)



(897, 1373)


Processing year 2001: 100%|██████████| 366/366 [00:51<00:00,  7.15it/s]


Data saved to daily_TG_average_night.nc


In [15]:
import numpy as np

def get_mask_corners(mask_data):
    # 找到所有非零元素的行列索引
    rows, cols = np.nonzero(mask_data!= 65535)
    
    # 找到最小和最大行列索引（左上角和右下角）
    top_left = (rows.min(), cols.min())      # 左上角
    top_right = (rows.min(), cols.max())     # 右上角
    bottom_left = (rows.max(), cols.min())   # 左下角
    bottom_right = (rows.max(), cols.max())  # 右下角
    
    return top_left, top_right, bottom_left, bottom_right
#mask_file = 'D:\CSSP\DKMD/mask_TG_DKMD.tif'
mask_file = 'D:/CSSP/SJY/mask_TGSJY5.tif'
# 示例用法

from rasterio.transform import rowcol, xy

with rasterio.open(mask_file) as dataset:
    mask_data = dataset.read(1)
    transform = dataset.transform
    
    # 获取四个角的像素坐标
    corners = get_mask_corners(mask_data)
    
    # 将像素索引转换为地理坐标
    geo_top_left = xy(transform, corners[0][0], corners[0][1])
    geo_top_right = xy(transform, corners[1][0], corners[1][1])
    geo_bottom_left = xy(transform, corners[2][0], corners[2][1])
    geo_bottom_right = xy(transform, corners[3][0], corners[3][1])
    print(corners )
    print("Mask Corners (Geographic Coordinates):")
    print(f"Top Left: {geo_top_left}")
    print(f"Top Right: {geo_top_right}")
    print(f"Bottom Left: {geo_bottom_left}")
    print(f"Bottom Right: {geo_bottom_right}")


((647, 1388), (647, 2760), (1543, 1388), (1543, 2760))
Mask Corners (Geographic Coordinates):
Top Left: (-1538432.4310474158, 4140996.6673463536)
Top Right: (-166432.43104741583, 4140996.6673463536)
Bottom Left: (-1538432.4310474158, 3244996.6673463536)
Bottom Right: (-166432.43104741583, 3244996.6673463536)


In [ ]:
import xarray as xr
ds = xr.open_dataset(daily_TG_average_night.nc)